Normalization

In [ ]:
import cv2
from moviepy.editor import *
import moviepy
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

min_height = 280
min_width = 520
max_width = 720
max_height = 420

for i in os.listdir("100words"):
    os.mkdir("ResizedVideos/" + i)

In [ ]:
count = 0
filenames = os.listdir("100words")
for filename in filenames:
    for i in os.listdir("100words/" + filename):
        try:
            clip = VideoFileClip("100words/" + filename + "/" + i)
            clip_size = []
            clip_size.append(clip.size[0])
            clip_size.append(clip.size[1])
            if clip_size[0]<min_width:
                clip_size[0] = min_width
                clip_size[1] = min_height
            elif clip_size[0] > max_width: 
                clip_size[0] = max_width
                clip_size[1] = max_height
            if clip_size[1]<min_height:
                clip_size[1] = min_height
                clip_size[0] = min_width
            elif clip_size[1] > max_height: 
                clip_size[1] = max_height   
                clip_size[0] = max_width
            x = clip.resize((clip_size[0],clip_size[1])).write_videofile("ResizedVideos/" + filename + "/" + str(count) + ".mp4" , fps = 30/clip.duration)
            count+=1
        except:
            pass

Augmentation

In [ ]:
import imgaug.augmenters as iaa
from PIL import Image
import cv2
import glob
from matplotlib import pyplot as plt
import os
import numpy as np
import time

def toVideo(aug , name , fps , size):
    out = cv2.VideoWriter(name,0, (fps), (size[1],size[0]))
    for i in range(len(aug)):
        out.write(aug[i][...,::-1])
    out.release()


def videotoframes(video_name):    
    frames = []
    # Opens the Video file
    cap= cv2.VideoCapture(video_name)
    fps = cap.get(cv2.CAP_PROP_FPS)
    i=0
    while(cap.isOpened()):
        #Read Frame
        ret,frame = cap.read()
        if ret == False:
            break
        i+=1
        frames.append(frame[...,::-1])
        
    cap.release()
    cv2.destroyAllWindows()
    return frames ,fps


def returnAugemntor(index , frames):
    augmentorList = [
        iaa.Affine(rotate=(10,10))(images=frames),
        iaa.Affine(rotate=(15,15))(images=frames),
        iaa.Affine(rotate=(-15,-15))(images=frames),
        iaa.Affine(rotate=(-10,-10))(images=frames),
        iaa.Affine(rotate=(20,20))(images=frames),
        iaa.Affine(rotate=(-20,-20))(images=frames),
        iaa.Affine(rotate=(30,30))(images=frames),
        iaa.Affine(rotate=(-30,-30))(images=frames),
        iaa.Affine(rotate=(40,40))(images=frames),
        iaa.Affine(rotate=(-40,-40))(images=frames)
        ]
    return augmentorList[index]

In [ ]:
directory = "ResizedVideos" 
LL = os.listdir(directory)
counter = 0
for foldername in LL:
    LL_NEW= os.listdir(directory+'/'+foldername)
    count = len(LL_NEW)  
    for augment in range(10):
        for filename in LL_NEW: 
            if  '_' not in filename:
                if count == 150:
                    break
                else:
                    print("Word : ", foldername ,  ", Count : ", counter, ", Toal : ", str(150000), ", Percentage Completed : ", str(counter/307900*100))
                    frames , fps = videotoframes(directory + '/' + foldername + '/' + filename)
                    size = frames[0].shape
                    aug = returnAugemntor(augment , frames)
                    name = directory + '/' + foldername + '/' + filename.split(".")[0]+"_" + str(count)+".mp4"
                    toVideo(aug , name , fps , size)
                    count+=1
                    counter+=1